# Data Preprocessing Tools

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!zip -r datasets.zip ./datasets

In [ ]:
!unzip ./datasets.zip

Archive:  ./datasets.zip
   creating: datasets/
   creating: datasets/animal/
   creating: datasets/animal/lion/
   creating: datasets/animal/lion/.ipynb_checkpoints/
   creating: datasets/animal/lion/Train/
  inflating: datasets/animal/lion/Train/0010.txt  
  inflating: datasets/animal/lion/Train/0027.txt  
  inflating: datasets/animal/lion/Train/0013.txt  
  inflating: datasets/animal/lion/Train/0031.txt  
  inflating: datasets/animal/lion/Train/0041.txt  
  inflating: datasets/animal/lion/Train/0019.txt  
  inflating: datasets/animal/lion/Train/0002.txt  
  inflating: datasets/animal/lion/Train/0005.txt  
  inflating: datasets/animal/lion/Train/0044.txt  
  inflating: datasets/animal/lion/Train/0017.txt  
  inflating: datasets/animal/lion/Train/0025.jpg  
  inflating: datasets/animal/lion/Train/0009.jpg  
  inflating: datasets/animal/lion/Train/0012.jpg  
  inflating: datasets/animal/lion/Train/0049.jpg  
  inflating: datasets/animal/lion/Train/0040.jpg  
  inflating: datasets/anima

In [ ]:
import os
import yaml

# Load the combined dataset configuration
config_path = 'combined_datasets.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

base_path = config['path']
classes = config['names']

# Function to create labels for images in a given directory
def create_labels(directory_path, class_name):
    class_id = classes.index(class_name)
    full_directory_path = os.path.join(base_path, directory_path)

    if not os.path.isdir(full_directory_path):
        print(f"Warning: Directory not found: {full_directory_path}")
        return

    print(f"Processing directory: {full_directory_path} for class: {class_name} (ID: {class_id})")

    # The label file should be in the same directory as the image file
    # No need to create a separate 'labels' subdirectory

    for filename in os.listdir(full_directory_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            # image_path = os.path.join(full_directory_path, filename) # Not strictly needed here
            label_filename = os.path.splitext(filename)[0] + '.txt'
            label_path = os.path.join(full_directory_path, label_filename) # Save label in the same directory

            # YOLO format: <class_id> <center_x> <center_y> <width> <height>
            # Assuming the object fills the whole image
            label_content = f"{class_id} 0.5 0.5 1.0 1.0"

            with open(label_path, 'w') as f:
                f.write(label_content)
            # print(f"Created label: {label_path}") # Uncomment for verbose output

# Process train directories
print("Generating training labels...")
for train_dir_entry in config['train']:
    # Determine the class name from the directory entry
    # This assumes the last part of the path is the class name
    class_name = os.path.basename(train_dir_entry).lower()
    if class_name == 'train' or class_name == 'val': # Skip generic 'train' or 'val' parts of the path
         # Need a more robust way to get class name, let's use the parent directory name if the last part is 'Train' or 'Val'
         parent_dir = os.path.dirname(train_dir_entry)
         class_name = os.path.basename(parent_dir).lower()

    # Handle specific cases based on the provided config structure
    if 'walk_or_run' in train_dir_entry:
        class_name = os.path.basename(train_dir_entry).lower() # 'run' or 'walk'
    elif 'data' in train_dir_entry:
        class_name = 'accident'
    elif 'animal' in train_dir_entry:
        # For the animal dataset, the class name is the folder *before* the Train/Val folder
        class_name = os.path.basename(os.path.dirname(train_dir_entry)).lower()


    if class_name in classes:
         create_labels(train_dir_entry, class_name)
    else:
        print(f"Warning: Could not determine class name for directory: {train_dir_entry}")


# Process validation directories
print("\nGenerating validation labels...")
for val_dir_entry in config['val']:
     # Determine the class name from the directory entry
    class_name = os.path.basename(val_dir_entry).lower()
    if class_name == 'train' or class_name == 'val': # Skip generic 'train' or 'val' parts of the path
         parent_dir = os.path.dirname(val_dir_entry)
         class_name = os.path.basename(parent_dir).lower()

    # Handle specific cases based on the provided config structure
    if 'walk_or_run' in val_dir_entry:
        class_name = os.path.basename(val_dir_entry).lower() # 'run' or 'walk'
    elif 'data' in val_dir_entry:
        class_name = 'accident'
    elif 'animal' in val_dir_entry:
        # For the animal dataset, the class name is the folder *before* the Train/Val folder
        class_name = os.path.basename(os.path.dirname(val_dir_entry)).lower()


    if class_name in classes:
        create_labels(val_dir_entry, class_name)
    else:
        print(f"Warning: Could not determine class name for directory: {val_dir_entry}")


print("\nLabel generation complete.")

Generating training labels...
Processing directory: /content/kaggle_datasets/walk_or_run_train/train/run for class: run (ID: 1)
Processing directory: /content/kaggle_datasets/walk_or_run_train/train/walk for class: walk (ID: 0)
Processing directory: /content/kaggle_datasets/data/train for class: accident (ID: 2)
Processing directory: /content/kaggle_datasets/animal/lion/Train for class: lion (ID: 3)

Generating validation labels...
Processing directory: /content/kaggle_datasets/walk_or_run_test/test/run for class: run (ID: 1)
Processing directory: /content/kaggle_datasets/walk_or_run_test/test/walk for class: walk (ID: 0)
Processing directory: /content/kaggle_datasets/data/val for class: accident (ID: 2)
Processing directory: /content/kaggle_datasets/animal/lion/Val for class: lion (ID: 3)

Label generation complete.


In [ ]:
import os

# Set up Kaggle API credentials
os.environ['KAGGLE_USERNAME'] = 'anasfathurrahman'
os.environ['KAGGLE_KEY'] = 'c5d1442d162f9fcdc69737096692bb63'

In [ ]:
# Download the datasets from Kaggle
!kaggle datasets download -d huan9huan/walk-or-run -p ./datasets --unzip
!kaggle datasets download -d ckay16/accident-detection-from-cctv-footage -p ./datasets --unzip
!kaggle datasets download -d chandrug/wildanimaldataset -p ./datasets --unzip

Dataset URL: https://www.kaggle.com/datasets/huan9huan/walk-or-run
License(s): CC0-1.0
walk-or-run.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/ckay16/accident-detection-from-cctv-footage
License(s): ODbL-1.0
accident-detection-from-cctv-footage.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/chandrug/wildanimaldataset
License(s): unknown
wildanimaldataset.zip: Skipping, found more recently modified local copy (use --force to force download)


## Merge Data Train ALL

In [ ]:
import os

# Get the absolute path to the datasets directory
datasets_path = os.path.abspath('./datasets')

# Create a combined dataset configuration file with absolute paths
combined_config = f"""
path: {datasets_path}
train:
  - walk_or_run_train/train/run
  - walk_or_run_train/train/walk
  - data/train
  - animal/lion/Train
val:
  - walk_or_run_test/test/run
  - walk_or_run_test/test/walk
  - data/val
  - animal/lion/Val

nc: 4
names: ['walk', 'run', 'accident', 'lion']
"""

with open('combined_datasets.yaml', 'w') as f:
    f.write(combined_config)

print("Created combined_datasets.yaml with absolute paths")

Created combined_datasets.yaml with absolute paths


## Training Test For Lion Animal

In [ ]:
import os
from glob import glob

# Path ke folder train dan val dari config yaml
train_dir = os.path.join(os.path.abspath('./datasets'), 'animal/lion/Train')
val_dir = os.path.join(os.path.abspath('./datasets'), 'animal/lion/Val')

def label_full_image(image_dir, class_id=0):
    image_files = glob(os.path.join(image_dir, '*.jpg')) + glob(os.path.join(image_dir, '*.jpeg')) + glob(os.path.join(image_dir, '*.png'))
    for img_path in image_files:
        label_path = os.path.splitext(img_path)[0] + '.txt'
        # YOLO format: <class_id> <center_x> <center_y> <width> <height>
        label_content = f"{class_id} 0.5 0.5 1.0 1.0"
        with open(label_path, 'w') as f:
            f.write(label_content)
        print(f"Label dibuat: {label_path}")

print('Melabeli gambar lion di Train...')
label_full_image(train_dir, class_id=0)
print('Melabeli gambar lion di Val...')
label_full_image(val_dir, class_id=0)
print('Selesai pelabelan otomatis untuk lion (full image bounding box).')

In [ ]:
import os

# Get the absolute path to the datasets directory
datasets_path = os.path.abspath('./datasets')

# Create a combined dataset configuration file with absolute paths
combined_config = f"""
path: {datasets_path}
train:
  - animal/lion/Train
val:
  - animal/lion/Val

nc: 1
names: ['lion']
"""

with open('combined_datasets.yaml', 'w') as f:
    f.write(combined_config)

print("Created combined_datasets.yaml with absolute paths")

Created combined_datasets.yaml with absolute paths


# Training Environtment Mode CPU/GPU
Pelatihan YoLo Model 12 menggunakan dataset
Belum dapat berhasil untuk melakukan training menggunakan TPU Google Colab

In [ ]:
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [ ]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLO12 model
model = YOLO("yolo12m.pt")

# Train the model on the combined dataset for a suitable number of epochs (e.g., 100)
results = model.train(data="combined_datasets.yaml", epochs=100, imgsz=640)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=combined_datasets.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              


  6                  -1  2   2689536  ultralytics.nn.modules.block.A2C2f           [512, 512, 2, True, 4]        
  7                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  8                  -1  2   2689536  ultralytics.nn.modules.block.A2C2f           [512, 512, 2, True, 1]        
  9                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 10             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 11                  -1  1   1248768  ultralytics.nn.modules.block.A2C2f           [1024, 512, 1, False, -1]     
 12                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 13             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 14                  -1  1    378624  ultralytics.nn.modules.block.A2C2f           [1024

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 11.6±10.3 MB/s, size: 10.3 KB)


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      11.3G      0.124     0.1999     0.9591          2        640: 100%|██████████| 4/4 [00:02<00:00,  1.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.03it/s]


                   all         10         10      0.994          1      0.995      0.985

100 epochs completed in 0.112 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 40.8MB
Optimizer stripped from runs/detect/train/weights/best.pt, 40.8MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12m summary (fused): 169 layers, 20,105,683 parameters, 0 gradients, 67.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.77it/s]


                   all         10         10      0.995          1      0.995      0.995
Speed: 1.1ms preprocess, 20.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train


In [ ]:
# Load Model
model_trained = YOLO("./runs/detect/train/weights/last.pt")

# Perform object detection on an image using the model
# results = model_trained("./datasets/animal/lion/Val/0051.jpg", conf=0.25, save=True)
results = model_trained("./lion-test2.jpg", conf=0.25, save=True)


image 1/1 /content/lion-test2.jpg: 352x640 1 lion, 53.8ms
Speed: 2.1ms preprocess, 53.8ms inference, 1.6ms postprocess per image at shape (1, 3, 352, 640)
Results saved to runs/detect/predict3


In [ ]:
# Print hasil deteksi secara detail
print('Results:', results)
if len(results) > 0:
    print('Boxes:', results[0].boxes)
    print('Classes:', results[0].boxes.cls if hasattr(results[0].boxes, 'cls') else None)
    print('Scores:', results[0].boxes.conf if hasattr(results[0].boxes, 'conf') else None)
else:
    print('Tidak ada deteksi.')

Results: [ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'lion'}
obb: None
orig_img: array([[[ 99,  99,  99],
        [100, 100, 100],
        [101, 101, 101],
        ...,
        [114, 112, 112],
        [114, 112, 112],
        [114, 112, 112]],

       [[ 99,  99,  99],
        [100, 100, 100],
        [101, 101, 101],
        ...,
        [114, 112, 112],
        [114, 112, 112],
        [114, 112, 112]],

       [[100, 100, 100],
        [100, 100, 100],
        [101, 101, 101],
        ...,
        [114, 112, 112],
        [114, 112, 112],
        [114, 112, 112]],

       ...,

       [[ 37,  52,  55],
        [ 38,  53,  56],
        [ 36,  52,  58],
        ...,
        [ 45,  67,  79],
        [ 42,  63,  78],
        [ 40,  61,  76]],

       [[ 37,  52,  55],
        [ 37,  52,  55],
        [ 36,  52,  58],
        ...,
        [ 53,  72,  85],
        [ 51,  70,  85],
     